# Initialize spark

In [1]:
from pyspark.sql.types import *
scc = StreamingContext(spark,4)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5,application_1655323491717_0006,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Initialize data streaming

### Initialize monitor data streaming

In [2]:
schema = StructType([
    StructField("device_id", StringType(), True),
    StructField("device_health", IntegerType(), True),
    StructField("type", StringType(), True),
    StructField("area", StringType(), True),
    StructField("customer", StringType(), True),
])
monitor_stream = spark.readStream.schema(schema).format("json").option('path','s3://guglielmo-data-lake/monitor').load()
monitor_stream

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[device_id: string, device_health: int, type: string, area: string, customer: string]

In [3]:
monitor_stream.isStreaming

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True

### Initialize sensors data streaming

In [4]:
schema = StructType([
    StructField("device_id", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("humidity", IntegerType(), True),
    StructField("temperature", IntegerType(), True)
])
t_sensors_stream = spark.readStream.schema(schema).format("json").option('path','s3://guglielmo-data-lake/measurements/*/p*').load()
t_sensors_stream

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[device_id: string, timestamp: timestamp, humidity: int, temperature: int]

In [5]:
schema = StructType([
    StructField("device_id", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("CO2_level", IntegerType(), True)
])
p_sensors_stream = spark.readStream.schema(schema).format("json").option('path','s3://guglielmo-data-lake/measurements/*/t*').load()
p_sensors_stream

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[device_id: string, timestamp: timestamp, CO2_level: int]

In [6]:
print(monitor_stream.isStreaming, t_sensors_stream.isStreaming, p_sensors_stream.isStreaming)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True True True

In [7]:
extract_test = monitor_stream.select("device_health")
extract_test.writeStream\
  .outputMode("append")\
  .format("json")\
  .option("path", "s3://test-output-emr/")\
  .option("checkpointLocation", "s3://test-spark-checkpoints/")\
  .start().awaitTermination()

'''extract_test = t_sensors_stream.select("humidity")
extract_test.writeStream\
  .outputMode("append")\
  .format("json")\
  .option("path", "s3://test-output-emr/")\
  .option("checkpointLocation", "s3://test-spark-checkpoints/")\
  .start().awaitTermination()

extract_test = monitor_stream.select("CO2_level")
extract_test.writeStream\
  .outputMode("append")\
  .format("json")\
  .option("path", "s3://test-output-emr/")\
  .option("checkpointLocation", "s3://test-spark-checkpoints/")\
  .start().awaitTermination()'''

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkStatementCancellationFailedException: Interrupted by user but Livy failed to cancel the Spark statement. The Livy session might have become unusable.